In [204]:
import pandas as pd

testcases = pd.read_csv('../data/mozilla_firefox/firefoxDataset/docs_english/TC/script/testcases.csv')
print('Test Cases Shape: {}'.format(testcases.shape))

bugreports_p1 = pd.read_csv('../data/mozilla_firefox/firefoxDataset/docs_english/BR_2/all_bugs_p1.csv', sep="|")
bugreports_p2 = pd.read_csv('../data/mozilla_firefox/firefoxDataset/docs_english/BR_2/all_bugs_p2.csv', sep='|')
bugreports_p3 = pd.read_csv('../data/mozilla_firefox/firefoxDataset/docs_english/BR_2/all_bugs_p3.csv', sep='|')

bugreports = pd.concat([bugreports_p1, bugreports_p2, bugreports_p3])
print('Bug Reports shape: {}'.format(br_df.shape))

Test Cases Shape: (207, 8)
Bug Reports shape: (35955, 10)


In [211]:
testcases.tail()

,TC_Number,TestDay,Gen_Title,Crt_Nr,Title,Preconditions,Steps,Expected_Result,tc_name,tc_desc
202,204,20161125 + 20170106,zoom indicator,20,Zoom Indicator & View>Zoom>Zoom Text Only,NaN,1. Launch Firefox with a clean profile\n\n2. E...,1. - Firefox is successfully launched\n- Defau...,TC_204_TRG,204 20161125 + 20170106 zoom indicator 20 Zoom...
203,205,20161125 + 20170106,zoom indicator,21,Zoom level in a Private Window functions same ...,NaN,1. Launch Firefox with a clean profile\n2. Ope...,1. - Firefox is successfully launched\n- Defau...,TC_205_TRG,205 20161125 + 20170106 zoom indicator 21 Zoom...
204,206,20161125 + 20170106,zoom indicator,21,Zoom level in a Private Window functions same ...,NaN,1. Launch Firefox with a clean profile\n2. Ope...,1. - Firefox is successfully launched\n- Defau...,TC_206_TRG,206 20161125 + 20170106 zoom indicator 21 Zoom...
205,207,20161125 + 20170106,zoom indicator,23,Zoom level in a Private Window functions same ...,NaN,1. Launch Firefox with a clean profile\n2. Ope...,1. - Firefox is successfully launched\n- Defau...,TC_207_TRG,207 20161125 + 20170106 zoom indicator 23 Zoom...
206,208,20161125 + 20170106,zoom indicator,24,Zoom level in a Private Window functions same ...,NaN,1. Launch Firefox with a clean profile\n2. Ent...,1. - Firefox is successfully launched\n- Defau...,TC_208_TRG,208 20161125 + 20170106 zoom indicator 24 Zoom...


In [206]:
bugreports.head()

,Bug_Number,Summary,Platform,Component,Version,Creation_Time,Whiteboard,QA_Whiteboard,First_Comment_Text,First_Comment_Creation_Time
0,506297,Livemarks with null site/feed uris cause sync ...,All,Sync,unspecified,2009-07-24T17:08:43Z,NaN,NaN,2009-07-24 09:54:28 FaultTolerance D...,2009-07-24T17:08:43Z
1,506338,Enhance Crash Recovery to better help the user,All,Session Restore,Trunk,2009-07-24T19:17:21Z,[crashkill][crashkill-metrics],NaN,When our users crash they are pretty much in t...,2009-07-24T19:17:21Z
2,506507,Dragging multiple bookmarks in the bookmarks s...,x86,Bookmarks & History,Trunk,2009-07-26T06:16:02Z,NaN,NaN,User-Agent: Mozilla/5.0 (Windows; U; Win...,2009-07-26T06:16:02Z
3,506550,Unreliable Back Button navigating nytimes.com,x86,Extension Compatibility,3.5 Branch,2009-07-26T16:12:10Z,[caused by adblock plus][platform-rel-NYTimes],NaN,User-Agent: Mozilla/5.0 (Windows; U; Win...,2009-07-26T16:12:10Z
4,506575,ALT + F4 when dropdown of autocomplete is open...,x86,Address Bar,3.5 Branch,2009-07-26T20:14:54Z,NaN,NaN,Pressing ALT + F4 when the autocomplete dropdo...,2009-07-26T20:14:54Z


#### Bug Reports Names and Descriptions

In [207]:
bugreports['br_name'] = bugreports.apply(lambda row : 'BR_' + str(row[0]) + '_SRC', axis=1)
bugreports['br_desc'] = bugreports.apply(lambda row : ' '.join([str(el) for el in row]), axis=1) 

#### Test Cases Names and Descriptions

In [208]:
testcases['tc_name'] = testcases.apply(lambda row : 'TC_' + str(row[0]) + '_TRG', axis=1)
testcases['tc_desc'] = testcases.apply(lambda row : ' '.join([str(el) for el in row]), axis=1)

#### Create Oracle

In [213]:
firefox_versions = ['48 Branch', '49 Branch', '50 Branch', '51 Branch']
map_fversion_to_testday = {
    '48 Branch' : ['20160603' '20160624' '20160708'],
    '49 Branch' : ['20160722' '20160812' '20160826'],
    '50 Branch' : ['20160909' '20160930' '20161014'],
    '51 Branch' : ['20161028' '20161125' '20170106']
}

links_df = pd.DataFrame(columns=['src_artf','trg_artf','link'])

counter = 1
for br in bugreports.iterrows():
    for tc in testcases.iterrows():
        for tday in tc['TestDay'].split(' + '):
            if tday in map_fversion_to_testday[br['Version']]:
                links_df.loc[counter] = [br['br_name'],tc['tc_name'],1]
                break
            else:
                links_df.loc[counter] = [br['br_name'],tc['tc_name'],0]
            
            counter = counter + 1

print('Expected instances amount: {}'.format(len(bugreports) * len(testcases)))

print('Num BRs 48 Branch: {}'.format(len(bugreports[bugreports.Version == '48 Branch'])))
print('Num BRs 49 Branch: {}'.format(len(bugreports[bugreports.Version == '49 Branch'])))
print('Num BRs 50 Branch: {}'.format(len(bugreports[bugreports.Version == '50 Branch'])))
print('Num BRs 51 Branch: {}'.format(len(bugreports[bugreports.Version == '51 Branch'])))

print('Num TestCases: {}'.format(len(testcases)))

TypeError: tuple indices must be integers or slices, not str